# ACOPF in Rectangular Form

ACOPF based on rectangular formulation
This is the 6 bus example from pp. 372-376 of
"Power Generation, Operation, and Control, 3nd Edition",
by Allen. J. Wood and Bruce F. Wollenberg, John Wiley & Sons, NY, Jan 1996.

In [2]:
sbase = 100 # MVA

# Cost Coefficents
a = [213.1, 200.0, 240.0,0,0,0]
b = [11.669, 10.333, 10.833, 0,0,0]*sbase
c = [0.00533, 0.00889, 0.00741, 0,0,0]*sbase^2

# Generation Limits
Pmin = [50, 37.5, 45,0,0,0]/sbase
Pmax = [200, 150, 180,0,0,0]/sbase

Qmin = [-100,-100,-100,0,0,0]/sbase
Qmax = [150,150,120,0,0,0]/sbase

# Load Information
Pload = [0,0,0,100,100,100]/sbase
Qload = [0,0,0,15,15,15]/sbase

# Voltage limits
Vmin = [0.95,0.95,0.95,0.95,0.95,0.95,0.95]
Vmax = [1.07,1.07,1.07,1.07,1.07,1.07]




B = [-11.74791555 4 0 4.705882353 3.112033195 0;
     4 -23.19549683 3.846153846 8 3 4.454342984;
     0 3.846153846 -16.56727017 0 3.170731707 9.615384615
     4.705882353 8 0 -14.63588235 2 0;
     3.112033195 3 3.170731707 2 -14.1377649 3
     0 4.454342984 9.615384615 0 3 -17.0047276 ]

G = [4.006346107 -2 0 -1.176470588 -0.829875519 0
     -2 9.328250814 -0.769230769 -4 -1 -1.559020045
      0 -0.769230769 4.155722326 0 -1.463414634 -1.923076923
      -1.176470588 -4 0 6.176470588 -1 0
      -0.829875519 -1 -1.463414634 -1 5.293290153 -1
       0 -1.559020045 -1.923076923 0 -1 4.482096968 ];


using JuMP, Ipopt

ACOPF = Model(solver=IpoptSolver())

nbus = 6

@variable(ACOPF, Pgen[1:nbus])  # Active power generation at bus i
@variable(ACOPF, Qgen[1:nbus])  # Reactive power generation at bus i
@variable(ACOPF, E[1:nbus], start = 1.07)     # Real part of the complex voltage at bus i
@variable(ACOPF, F[1:nbus], start = 0)     # Reactive part of the complex voltage at bus i

# Power Flow equations :
@NLconstraint(ACOPF, Act_Pwr_Blnc[i=1:nbus],
              Pgen[i] - Pload[i] == sum( G[i,j]*(E[i]*E[j]+F[i]*F[j]) + B[i,j]*(F[i]*E[j]-E[i]*F[j]) for j=1:nbus ) )

@NLconstraint(ACOPF, React_Pwr_Blnc[i=1:nbus],
              Qgen[i] - Qload[i] == sum( G[i,j]*(F[i]*E[j]-E[i]*F[j]) - B[i,j]*(E[i]*E[j]+F[i]*F[j]) for j=1:nbus ) )


# Limits of active and reactive power : 
@constraint(ACOPF, Act_Pwr_Gen_Lim[i=1:nbus],   Pmin[i] <= Pgen[i] <= Pmax[i] )
@constraint(ACOPF, React_Pwr_Gen_Lim[i=1:nbus], Qmin[i] <= Qgen[i] <= Qmax[i] )

# Limits of voltage at each bus : 
@NLconstraint(ACOPF, Voltage_Mag_Lim[i=1:nbus], Vmin[i]*Vmin[i] <= E[i]*E[i]+F[i]*F[i] <= Vmax[i]*Vmax[i] )

# Constraint of reference bus :
@constraint(ACOPF, Slack_Bus_Cons_1, E[1] == 1.07 )
@constraint(ACOPF, Slack_Bus_Cons_2, F[1] == 0 )


@objective(ACOPF, Min, sum( c[i]*Pgen[i]*Pgen[i] + b[i]*Pgen[i] + a[i] for i=1:nbus ))


status = solve(ACOPF)


println(" ---------------------------- ")
println( "    Real Dispatches = ", getvalue(Pgen*sbase) )
println( "Reactive Dispatches = ", getvalue(Qgen*sbase) )
println( "       Bus Voltages = ", [sqrt(getvalue(E[i]*E[i]+F[i]*F[i])) for i in 1:nbus ] )
println( " Objective Func. Value = ", getobjectivevalue(ACOPF) )

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      164
Number of nonzeros in inequality constraint Jacobian.:       18
Number of nonzeros in Lagrangian Hessian.............:      414

Total number of variables............................:       24
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       20
Total number of inequality constraints...............:       12
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:       12
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 